## Resolve dependências

In [1]:
!pip install -q albumentations==1.4.6 opencv-python pycocotools tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q pycocotools albumentations==1.4.6 opencv-python tqdm
!pip uninstall -y sympy
!pip install sympy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 9.5 MB/s eta 0:00:00
Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.8 MB/s eta 0:00:00


In [2]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.24.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 694356, done.
remote: Counting objects: 100% (1694/1694), done.
remote: Compressing objects: 100% (880/880), done.
remote: Total 694356 (delta 1525), reused 887 (delta 806), pack-reused 692662 (from 3)
Receiving objects: 100% (694356/694356), 1.22 GiB | 32.70 MiB/s, done.
Resolving deltas: 100% (650252/650252), done.
fatal: not a git repository (or any of the parent directories): .git


In [3]:
import sys
# sys.path.append("/content/drive/MyDrive/TCC/src/")
sys.path.append("/content/drive/MyDrive/TCC/notebooks/Faster R-CNN")

In [4]:
from pathlib import Path

DATASET_ROOT = Path("/content/drive/MyDrive/TCC/DATASET")
paths = {
    "bh_watertanks": {
        "images": {
            "train": DATASET_ROOT/"bh_watertanks/images/train",
            "val":   DATASET_ROOT/"bh_watertanks/images/val",
            "test":  DATASET_ROOT/"bh_watertanks/images/test",
        },
        "ann": {
            "train": DATASET_ROOT/"bh_watertanks/annotations/instances_train.json",
            "val":   DATASET_ROOT/"bh_watertanks/annotations/instances_val.json",
            "test":  DATASET_ROOT/"bh_watertanks/annotations/instances_test.json",
        }
    }
}

In [5]:
import sys
from pathlib import Path
sys.path.append("/content/drive/MyDrive/TCC/src/")
DATASET_ROOT = Path("/content/drive/MyDrive/TCC/campinas_dataset")

campinas_paths = {
    "bh_watertanks": {
        "images": {
            "train": DATASET_ROOT/"train/images/",
            "test":  DATASET_ROOT/"test/images/",
        },
        "ann": {
            "train": DATASET_ROOT/"train/annotations/instances_train.json",
            "test":  DATASET_ROOT/"test/annotations/instances_test.json",
        }
    }
}

## Carrega Loaders

In [6]:
from my_utils.coco_dataset import CocoDetDataset
from my_utils.my_utils import get_transforms, collate_fn, evaluate_and_visualize
from torch.utils.data import Dataset, DataLoader


watertanks_train = CocoDetDataset(
    paths["bh_watertanks"]["images"]["train"],
    paths["bh_watertanks"]["ann"]["train"],
    transforms=get_transforms(train=True))

watertanks_val   = CocoDetDataset(
    paths["bh_watertanks"]["images"]["val"],
    paths["bh_watertanks"]["ann"]["val"],
    transforms=get_transforms(train=False))

watertanks_test  = CocoDetDataset(
    paths["bh_watertanks"]["images"]["test"],
    paths["bh_watertanks"]["ann"]["test"],
    transforms=get_transforms(train=False))


watertanks_train_loader = DataLoader(watertanks_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
watertanks_val_loader   = DataLoader(watertanks_val,   batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
watertanks_test_loader  = DataLoader(watertanks_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

loading annotations into memory...
Done (t=5.10s)
creating index...
index created!
loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
loading annotations into memory...
Done (t=0.60s)
creating index...
index created!


In [7]:
from my_utils.coco_dataset import CocoDetDataset
from my_utils.my_utils import get_transforms, collate_fn, evaluate_and_visualize
from torch.utils.data import Dataset, DataLoader


cmp_watertanks_train = CocoDetDataset(
    campinas_paths["bh_watertanks"]["images"]["train"],
    campinas_paths["bh_watertanks"]["ann"]["train"],
    transforms=get_transforms(train=True))

cmp_watertanks_test  = CocoDetDataset(
    campinas_paths["bh_watertanks"]["images"]["test"],
    campinas_paths["bh_watertanks"]["ann"]["test"],
    transforms=get_transforms(train=False))


cmp_watertanks_train_loader = DataLoader(cmp_watertanks_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
cmp_watertanks_test_loader  = DataLoader(cmp_watertanks_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

loading annotations into memory...
Done (t=1.07s)
creating index...
index created!
loading annotations into memory...
Done (t=1.01s)
creating index...
index created!


## Inicializa model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class WindowMHSA(nn.Module):
    def __init__(self, dim, num_heads=4, window_size=7, dim_reduction=None, dropout=0.0):
        super().__init__()
        self.window_size = window_size
        self.in_dim = dim
        self.mid_dim = dim_reduction or dim  # opcional reduzir canais
        self.pre = nn.Conv2d(dim, self.mid_dim, 1, bias=False) if self.mid_dim != dim else nn.Identity()
        self.norm1 = nn.LayerNorm(self.mid_dim)
        self.attn = nn.MultiheadAttention(self.mid_dim, num_heads, dropout=dropout, batch_first=True)
        # FFN
        mlp_hidden = int(self.mid_dim * 4)
        self.mlp = nn.Sequential(
            nn.Linear(self.mid_dim, mlp_hidden),
            nn.GELU(),
            nn.Linear(mlp_hidden, self.mid_dim),
        )
        self.norm2 = nn.LayerNorm(self.mid_dim)
        self.post = nn.Conv2d(self.mid_dim, dim, 1, bias=False) if self.mid_dim != dim else nn.Identity()

    def _pad_to_window(self, x):
        B, C, H, W = x.shape
        ws = self.window_size
        padH = (ws - H % ws) % ws
        padW = (ws - W % ws) % ws
        if padH or padW:
            x = F.pad(x, (0, padW, 0, padH))
        return x, H, W, padH, padW

    def forward(self, x):  # x: [B, C, H, W]
        B, C, H, W = x.shape
        y = self.pre(x)
        y, oH, oW, padH, padW = self._pad_to_window(y)
        B, C2, H2, W2 = y.shape
        ws = self.window_size

        # [B, C2, H2, W2] -> janelas: [B*nw, ws*ws, C2]
        y = y.unfold(2, ws, ws).unfold(3, ws, ws)  # [B, C2, nH, nW, ws, ws]
        nH, nW = y.shape[2], y.shape[3]
        y = y.permute(0,2,3,1,4,5).contiguous().view(B*nH*nW, C2, ws*ws)
        y = y.permute(0,2,1).contiguous()  # [B*nw, L, C2]

        # MHSA por janela
        y = self.norm1(y)
        attn_out, _ = self.attn(y, y, y)  # [B*nw, L, C2]
        y = y + attn_out
        y = y + self.mlp(self.norm2(y))

        # volta para [B, C2, H2, W2]
        y = y.permute(0,2,1).contiguous().view(B, nH, nW, C2, ws, ws)
        y = y.permute(0,3,1,4,2,5).contiguous().view(B, C2, nH*ws, nW*ws)
        if padH or padW:
            y = y[:, :, :oH, :oW]

        y = self.post(y)
        return x + y  # residual


In [9]:
class BackboneWithWindowAttn(nn.Module):
    def __init__(self, backbone, attn_levels=('3','4'), num_heads=4, window=7, dim_reduction=None):
        super().__init__()
        self.backbone = backbone
        self.attn_levels = set(attn_levels)
        self.attn = nn.ModuleDict()
        self._initialized = False
        self._cfg = dict(num_heads=num_heads, window=window, dim_reduction=dim_reduction)

    @torch.no_grad()
    def _init_for_shapes(self, feats: OrderedDict):
        for k, v in feats.items():
            if k in self.attn_levels:
                C = v.shape[1]
                mod = WindowMHSA(
                    dim=C,
                    num_heads=self._cfg['num_heads'],
                    window_size=self._cfg['window'],
                    dim_reduction=self._cfg['dim_reduction']
                )
                # >>> chave do conserto: mover o módulo pro device do feature map
                mod = mod.to(v.device)
                self.attn[k] = mod
        self._initialized = True

    def forward(self, x):
        feats = self.backbone(x)
        if not self._initialized:
            self._init_for_shapes(feats)
        out = OrderedDict()
        for k, v in feats.items():
            if k in self.attn_levels:
                out[k] = self.attn[k](v)   # agora mesmos devices
            else:
                out[k] = v
        return out


In [16]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


num_classes = 1 + 2

weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
base = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
base.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

base.backbone = BackboneWithWindowAttn(
    base.backbone,
    attn_levels=('3','4'),
    num_heads=4,
    window=7,
    dim_reduction=256
)

model = base.to(device)
model = model.to(device)

imgs0, _ = next(iter(cmp_watertanks_train_loader))
img0 = imgs0[0].to(device)

model.eval()
with torch.no_grad():
    _ = model([img0])

model.train()
for p in model.backbone.backbone.parameters():
    p.requires_grad = False

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=5e-4)


In [17]:
from engine import train_one_epoch, evaluate
for p in model.backbone.backbone.parameters():
    p.requires_grad = False
for name, m in model.backbone.named_modules():
    pass

for p in model.roi_heads.parameters():
    p.requires_grad = True
for p in model.rpn.parameters():
    p.requires_grad = True

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 12
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    if epoch == 2:
        for p in model.backbone.backbone.parameters():
            p.requires_grad = True
        new_params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(new_params, lr=0.002, momentum=0.9, weight_decay=5e-4)

    train_one_epoch(model, optimizer, cmp_watertanks_train_loader, device, epoch, print_freq=10)
    scheduler.step()


/content/engine.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [ 0/51]  eta: 0:01:08  lr: 0.000105  loss: 1.2538 (1.2538)  loss_classifier: 1.1334 (1.1334)  loss_box_reg: 0.0375 (0.0375)  loss_objectness: 0.0823 (0.0823)  loss_rpn_box_reg: 0.0006 (0.0006)  time: 1.3455  data: 1.1859  max mem: 4764
Epoch: [0]  [10/51]  eta: 0:00:15  lr: 0.001104  loss: 1.0239 (0.9212)  loss_classifier: 0.8673 (0.7664)  loss_box_reg: 0.0375 (0.0635)  loss_objectness: 0.0640 (0.0871)  loss_rpn_box_reg: 0.0031 (0.0041)  time: 0.3858  data: 0.2302  max mem: 4764
Epoch: [0]  [20/51]  eta: 0:00:12  lr: 0.002103  loss: 0.3487 (0.6118)  loss_classifier: 0.1853 (0.4530)  loss_box_reg: 0.0501 (0.0601)  loss_objectness: 0.0640 (0.0942)  loss_rpn_box_reg: 0.0042 (0.0045)  time: 0.3405  data: 0.1852  max mem: 4764
Epoch: [0]  [30/51]  eta: 0:00:08  lr: 0.003102  loss: 0.2093 (0.4966)  loss_classifier: 0.1100 (0.3480)  loss_box_reg: 0.0501 (0.0575)  loss_objectness: 0.0688 (0.0862)  loss_rpn_box_reg: 0.0040 (0.0048)  time: 0.4474  data: 0.2923  max mem: 4764
Epoch: [

In [ ]:
checkpoint_path = '/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/model/wt_t_.pth'
state = torch.load(checkpoint_path)
model.load_state_dict(state)

<All keys matched successfully>

## Treina modelo

In [ ]:
from torch.cuda.amp import GradScaler
scaler = GradScaler()

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.002,
    momentum=0.9,
    weight_decay=5e-4
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=10,
    eta_min=1e-5
)


/tmp/ipython-input-900805923.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
from engine import train_one_epoch
import torch

epochs = 10

for epoch in range(epochs):
  for p in model.backbone.body.parameters():
    p.requires_grad = True
    new_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(new_params, lr=0.002, momentum=0.9, weight_decay=5e-4)

  train_one_epoch(model, optimizer, cmp_watertanks_train_loader, device, epoch, print_freq=10)
  lr_scheduler.step()



/content/engine.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [ 0/51]  eta: 0:03:16  lr: 0.000042  loss: 0.5330 (0.5330)  loss_classifier: 0.2900 (0.2900)  loss_box_reg: 0.0046 (0.0046)  loss_objectness: 0.2261 (0.2261)  loss_rpn_box_reg: 0.0123 (0.0123)  time: 3.8482  data: 1.0665  max mem: 3727
Epoch: [0]  [10/51]  eta: 0:00:28  lr: 0.000442  loss: 0.5003 (0.4853)  loss_classifier: 0.2235 (0.2335)  loss_box_reg: 0.0032 (0.0042)  loss_objectness: 0.2261 (0.2316)  loss_rpn_box_reg: 0.0073 (0.0160)  time: 0.6841  data: 0.1940  max mem: 3913
Epoch: [0]  [20/51]  eta: 0:00:19  lr: 0.000841  loss: 0.3572 (0.3802)  loss_classifier: 0.1227 (0.1467)  loss_box_reg: 0.0032 (0.0071)  loss_objectness: 0.2089 (0.2110)  loss_rpn_box_reg: 0.0044 (0.0155)  time: 0.4762  data: 0.2155  max mem: 3913
Epoch: [0]  [30/51]  eta: 0:00:12  lr: 0.001241  loss: 0.1976 (0.3089)  loss_classifier: 0.0391 (0.1133)  loss_box_reg: 0.0122 (0.0112)  loss_objectness: 0.1204 (0.1709)  loss_rpn_box_reg: 0.0032 (0.0135)  time: 0.5222  data: 0.2613  max mem: 3913
Epoch: [

/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch: [1]  [ 0/51]  eta: 0:00:20  lr: 0.002000  loss: 0.1842 (0.1842)  loss_classifier: 0.0476 (0.0476)  loss_box_reg: 0.0981 (0.0981)  loss_objectness: 0.0288 (0.0288)  loss_rpn_box_reg: 0.0097 (0.0097)  time: 0.3944  data: 0.1308  max mem: 3913
Epoch: [1]  [10/51]  eta: 0:00:11  lr: 0.002000  loss: 0.1397 (0.1500)  loss_classifier: 0.0370 (0.0379)  loss_box_reg: 0.0823 (0.0890)  loss_objectness: 0.0131 (0.0151)  loss_rpn_box_reg: 0.0058 (0.0080)  time: 0.2758  data: 0.0122  max mem: 3913
Epoch: [1]  [20/51]  eta: 0:00:08  lr: 0.002000  loss: 0.1200 (0.1344)  loss_classifier: 0.0273 (0.0323)  loss_box_reg: 0.0792 (0.0841)  loss_objectness: 0.0100 (0.0124)  loss_rpn_box_reg: 0.0034 (0.0056)  time: 0.2644  data: 0.0003  max mem: 3913
Epoch: [1]  [30/51]  eta: 0:00:05  lr: 0.002000  loss: 0.1100 (0.1299)  loss_classifier: 0.0230 (0.0308)  loss_box_reg: 0.0731 (0.0828)  loss_objectness: 0.0081 (0.0110)  loss_rpn_box_reg: 0.0026 (0.0054)  time: 0.2647  data: 0.0003  max mem: 3913
Epoch: [

## Avalia Modelo

In [18]:
from my_utils.my_utils import evaluate_ap_multi_iou
import numpy as np

csv_path = "/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/results_attention_layer_cmp.csv"
df_results = evaluate_ap_multi_iou(
    model=model,
    loader=cmp_watertanks_test_loader,
    device=device,
    score_thr=0.25,
    iou_list=(0.50, 0.75, 0.90),
    iou_grid=np.arange(0.50, 0.96, 0.05),
    out_csv_path=csv_path
)

==== Resultados por classe ====
Classe 1 | mAP[.50:.95]=0.436 | AP@0.50=0.897 | AP@0.75=0.315 | AP@0.90=0.009 (GT=50)

📊 CSV salvo em: /content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/results_attention_layer_cmp.csv


In [19]:
torch.save(model.state_dict(), '/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/model/wt_t_att_cmp.pth')

In [20]:
from my_utils.my_utils import show_predictions

show_predictions(
    model=model,
    loader=watertanks_test_loader,
    device=device,
    n=2,
    score_thr=0.25
)


Output hidden; open in https://colab.research.google.com to view.